In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

df = pd.read_csv('diabetes_cleaned.csv')

In [43]:
#This function is specifically to categorize diagnosis ICD9 codes into buckets (Diabetes gets its own bucket)
#https://en.wikipedia.org/wiki/List_of_ICD-9_codes

def convert_icd9_codes(x):
    #Deal with string cases that would prevent conversion to float
    if x=='?':
        return('?')
    if x[0]=='V' or x[0]=='E':
        return('Supplementary')
    # Convert to float and mark diabetes as a special category
    x = float(x)
    if x>=249 and x<251:
        return('Diabetes')
    #Return the type of ICD9 code
    if x>=0 and x<=139:
        return('Infectious')
    if x>=140 and x<=239:
        return('Neoplasms')
    if x>=240 and x<=279:
        return('Endocrine_Immunity')
    if x>=280 and x<=289:
        return('Blood')
    if x>=290 and x<=319:
        return('Mental_Disorders')
    if x>=320 and x<=389:
        return('Nervous_System')
    if x>=390 and x<=459:
        return('Circulatory')
    if x>=460 and x<=519:
        return('Respiratory')
    if x>=520 and x<=579:
        return('Digestive')
    if x>=580 and x<=629:
        return('Genitourinary')
    if x>=630 and x<=679:
        return('Pregnancy')
    if x>=680 and x<=709:
        return('Skin')
    if x>=710 and x<=739:
        return('Muscles_Skeleton')
    if x>=740 and x<=759:
        return('Congenital')
    if x>=760 and x<=779:
        return('Perinatal')
    if x>=780 and x<=799:
        return('Generalized_Symptoms')
    if x>=800 and x<=999:
        return('Injury_Poison')
    else:
        return(None)

In [36]:
#Drop three patients with Gender not specified (they're also missing race and they're all not readmitted which seems weird)
df.drop(df.loc[df.gender=='Unknown/Invalid'].index,inplace=True)
#Dropping just identifier numbers since they have no information. Unnamed: 0 is just a redundant index column.
df.drop(columns=['encounter_id','patient_nbr','Unnamed: 0'],inplace=True)
#Weight, payer_code, and medical_specialty are dropped because they have so much missingness, per the authors.
df.drop(columns=['weight','payer_code','medical_specialty'],inplace=True)
#These medication columns are dropped because they are the same for all patients and have no information.
df.drop(columns=['examide','citoglipton','glimepiride-pioglitazone'],inplace=True)
#These medication columns have very few, but not zero, numbers of patients with information (less than 30).
df.drop(columns=['acetohexamide','tolbutamide','miglitol','troglitazone','tolazamide','glipizide-metformin','metformin-rosiglitazone','metformin-pioglitazone'],inplace=True)
#These mediations columns have between 30-500 patients, so Steady, Up, and Down are grouped together as 1, and No as 0.
#chlorpropamide, acarbose,nateglinide, glyburide-metformin (treated with drug_dict)

#Clean up dictionaries, indicating which strings are coded to which numbers
gender_dict = {'Female':0,'Male':1}
age_dict = {'[0-10)':0,'[10-20)':10,'[20-30)':20,'[30-40)':30,'[40-50)':40,'[50-60)':50,
           '[60-70)':60,'[70-80)':70,'[80-90)':80,'[90-100)':90}
readmitted_dict = {'NO':0,'>30':1,'<30':2}
drug_dict= {'No':0,'Steady':1,'Up':1,'Down':1}
change_dict= {'No':0,'Ch':1}
diabetesmed_dict = {'No':0,'Yes':1}

#Final dictionary indcating which columns to treat with which dictionary
cleanup_dict ={'gender':gender_dict,
               'age':age_dict,
               'chlorpropamide':drug_dict,
               'acarbose':drug_dict,
               'nateglinide':drug_dict,
               'glyburide-metformin':drug_dict,
              'readmitted':readmitted_dict,
              'change':change_dict,
              'diabetesMed':diabetesmed_dict}

#Replace the string/object values with numeric.
df.replace(to_replace=cleanup_dict,value=None,inplace=True)

#Convert ICD-9 codes to categories
df['diag_1'] = df.diag_1.apply(convert_icd9_codes)
df['diag_2'] = df.diag_2.apply(convert_icd9_codes)
df['diag_3'] = df.diag_3.apply(convert_icd9_codes)

#Function to specify a given column, make some dummies and drop the most abundant category
def dummify_and_drop_max(df,column):
    dummies = pd.get_dummies(data=df[column],prefix=column)
    to_drop = str(df[column].value_counts().index[0])
    dummies = dummies.drop(columns=[column+'_'+to_drop])
    return(dummies)

#Which columns to dummify
columns_to_dummify = ['race','admission_type_id','discharge_disposition_id','admission_source_id',
                      'max_glu_serum','A1Cresult','metformin','repaglinide','glimepiride',
                      'glipizide','glyburide','pioglitazone','rosiglitazone','insulin',
                     'diag_1','diag_2','diag_3',]

#Simple loop to get dummies, concat them to the end of the dataframe, and drop the original column so 
#there's no redundant information.
for i in columns_to_dummify:
    dummies = dummify_and_drop_max(df,i)
    df = pd.concat([df,dummies],axis=1)
    df.drop(columns=[i],inplace=True)

# Clean up the integer index to re-order, after deleting a few rows 
df.reset_index(drop=True,inplace=True)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71050 entries, 0 to 71049
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                71050 non-null  int64 
 1   encounter_id              71050 non-null  int64 
 2   patient_nbr               71050 non-null  int64 
 3   race                      71050 non-null  object
 4   gender                    71050 non-null  object
 5   age                       71050 non-null  object
 6   weight                    71050 non-null  object
 7   admission_type_id         71050 non-null  int64 
 8   discharge_disposition_id  71050 non-null  int64 
 9   admission_source_id       71050 non-null  int64 
 10  time_in_hospital          71050 non-null  int64 
 11  payer_code                71050 non-null  object
 12  medical_specialty         71050 non-null  object
 13  num_lab_procedures        71050 non-null  int64 
 14  num_procedures        

In [38]:
df.to_csv('diabetes_cleaned_numeric.csv')